# 멀티 모달: 이미지 캡셔닝
```
핵심 구성 요소
    - 이미지 인코더: 이미지의 시각적인 특징을 추출
    - 언어 디코더: 추출된 특징을 바탕으로 문장을 생성
```

## 이미지 인코더
```
이미지를 분석해서 중요한 시각적 정보 추출
사용되는 기술: CNN, ViT( Vision Transformer )
    CNN: 이미지의 픽셀 데이터를 기반. 물체, 색상, 텍스쳐, 위치 등의 특징을 추출
    ViT: 이미지를 패치 단위로 나누어서 Transformer 구조로 처리 -> 추세
```

## 언어 디코더
```
이미지의 특징을 받아서 자연스러운 문장을 생성 - 시계열 데이터처럼 처리
사용 기술: RNN, LSTM, Transformer
확률 분포를 이용해서 단어를 하나씩 선택해 문장을 생성
```

In [ ]:
# 리눅스에 패키지 설치. 특정 cuDNN 버전 설치
!apt install -allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

In [1]:
%pip uninstall -y tensorflow estimator keras

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install einops

  Using cached einops-0.8.1-py3-none-any.whl.metadata (13 kB)
Using cached einops-0.8.1-py3-none-any.whl (64 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
# 필요 라이브러리 설치
%pip install -U tensorflow_text tensorflow tensorflow_datasets

  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached h5py-3.13.0-cp310-cp310-win_amd64.whl.metadata (2.5 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl.metadata (1.9 kB)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-3.19.6-cp310-cp310-win_amd64.whl.metadata (806 bytes)
  Using cached wrapt-1.17.2-cp310-cp310-win_amd64.whl.metadata (6.5 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp310-cp310-win_amd64.whl.metadata (14 kB)
  Using cached grpcio-1.71.0-cp310-cp310-win_amd64.whl.metadata (4.0 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached requests-2.3

  DEPRECATION: Building 'promise' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'promise'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [ ]:
import pathlib
import tensorflow as tf
# 데이터 다운로드용 함수
def download_data(path="flickr8k"):
    path = pathlib.Path(path) # 파일 시스템 경로를 객체화. 문자열로 경로를 인식하게 하지 않아 다양한 os에서 호환이 가능하게 함.
    if len(list(path.rglob("*"))) < 16197:
        